In [1]:
# --- Install semua library eksternal ---
!pip install gensim --quiet
!pip install scikit-learn --quiet
!pip install nltk --quiet
!pip install matplotlib seaborn --quiet
!pip install tensorflow --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 531.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.8 MB/s eta 0:00:00


In [ ]:
!pip install numpy --upgrade --force-reinstall --quiet
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 90.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.


In [ ]:
# Fix masalah kompatibilitas numpy
!pip install numpy==1.23.5 --quiet
!pip install gensim --force-reinstall --no-cache-dir --quiet
import os
os.kill(os.getpid(), 9)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 57.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but yo

## Langkah 1: Load CSV & Labeling Otomatis

In [1]:
# Import library yang diperlukan
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load data hasil scraping
df = pd.read_csv('tiktok_reviews.csv')

# Bersihkan data kosong
df.dropna(subset=['content', 'score'], inplace=True)

# Labeling otomatis berdasarkan skor
def label_score(score):
    if score <= 2:
        return 'negative'
    elif score == 3:
        return 'neutral'
    else:
        return 'positive'

df['label'] = df['score'].apply(label_score)

# Cek distribusi kelas
print(df['label'].value_counts())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


label
positive    67500
negative    26000
neutral      6500
Name: count, dtype: int64


## Langkah 2: Preprocessing Teks

In [2]:
# Membersihkan teks
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)  # hapus newline
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # hapus link
    text = re.sub(r'\@w+|\#','', text)  # hapus mention & hashtag
    text = re.sub(r'[^A-Za-z\s]', '', text)  # hapus angka dan simbol
    text = re.sub(r'\s+', ' ', text).strip()  # hapus spasi berlebih
    return text

df['clean_text'] = df['content'].apply(lambda x: clean_text(str(x)))

# Hapus stopwords
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

df['clean_text'] = df['clean_text'].apply(remove_stopwords)
df[['content', 'clean_text', 'label']].head()


,content,clean_text,label
0,TikTok video 📷📸📷,tiktok video,negative
1,Ydo,ydo,positive
2,@khaskhelisb6,khaskhelisb,positive
3,@farissajid804 are very famous tiktoker you fo...,farissajid famous tiktoker follow id enjoy mom...,positive
4,Tik took,tik took,negative


## Langkah 3: Encode Label & Split Data

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode label
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])  # positive=2, neutral=1, negative=0

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'], df['label_encoded'],
    test_size=0.2, stratify=df['label_encoded'],
    random_state=42
)

print("Training data:", len(X_train))
print("Testing data :", len(X_test))


Training data: 80000
Testing data : 20000


## Langkah 4 : Tokenizing & Padding

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


MAX_NUM_WORDS = 20000
MAX_SEQUENCE_LENGTH = 100

# Tokenizer
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train)

# Convert to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH)

# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)


## Langkah 5 : Word2Vec Embedding Matrix

In [5]:
from gensim.models import Word2Vec
import numpy as np

# Train Word2Vec dari teks training
sentences = [text.split() for text in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
word_index = tokenizer.word_index

embedding_matrix = np.zeros((MAX_NUM_WORDS, 100))
for word, i in word_index.items():
    if i < MAX_NUM_WORDS:
        if word in w2v_model.wv:
            embedding_matrix[i] = w2v_model.wv[word]

## Langkah 6 : Pelatihan

### Skema Pelatihan 1: LSTM + Word2Vec + Split 80/20

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model1 = Sequential()
model1.add(Embedding(MAX_NUM_WORDS, 100, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model1.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(3, activation='softmax'))

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

history1 = model1.fit(X_train_pad, y_train_cat, epochs=5, batch_size=512, validation_data=(X_test_pad, y_test_cat))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │       2,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,000,000 (7.63 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,000,000 (7.63 MB)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.8437 - loss: 0.4460 - val_accuracy: 0.9610 - val_loss: 0.1032
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.9596 - loss: 0.1087 - val_accuracy: 0.9610 - val_loss: 0.1003
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.9588 - loss: 0.1080 - val_accuracy: 0.9610 - val_loss: 0.1000
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.9581 - loss: 0.1078 - val_accuracy: 0.9604 - val_loss: 0.0998
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.9599 - loss: 0.1043 - val_accuracy: 0.9610 - val_loss: 0.0996


#### Evaluasi Skema Pelatihan 1

In [8]:
from sklearn.metrics import accuracy_score, classification_report

y_pred1 = model1.predict(X_test_pad)
y_pred1_classes = np.argmax(y_pred1, axis=1)

print("Accuracy Skema 1:", accuracy_score(y_test, y_pred1_classes))
print(classification_report(y_test, y_pred1_classes, target_names=le.classes_))

625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step
Accuracy Skema 1: 0.96105
              precision    recall  f1-score   support

    negative       1.00      0.87      0.93      5200
     neutral       1.00      0.93      0.96      1300
    positive       0.95      1.00      0.97     13500

    accuracy                           0.96     20000
   macro avg       0.98      0.93      0.95     20000
weighted avg       0.96      0.96      0.96     20000



### Skema Pelatihan 2: LSTM + TF-IDF + Split 80/20

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

# LSTM but we reshape to 3D
X_train_tfidf_3d = np.expand_dims(X_train_tfidf, axis=2)
X_test_tfidf_3d = np.expand_dims(X_test_tfidf, axis=2)

model2 = Sequential()
model2.add(LSTM(128, input_shape=(X_train_tfidf_3d.shape[1], 1)))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(3, activation='softmax'))

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history2 = model2.fit(X_train_tfidf_3d, y_train_cat, epochs=5, batch_size=512, validation_data=(X_test_tfidf_3d, y_test_cat))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1015s 6s/step - accuracy: 0.6615 - loss: 0.8690 - val_accuracy: 0.6750 - val_loss: 0.7920
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1020s 6s/step - accuracy: 0.6754 - loss: 0.7966 - val_accuracy: 0.6750 - val_loss: 0.7895
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1047s 6s/step - accuracy: 0.6742 - loss: 0.7973 - val_accuracy: 0.6750 - val_loss: 0.7877
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 994s 6s/step - accuracy: 0.6756 - loss: 0.7900 - val_accuracy: 0.6797 - val_loss: 0.7753
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1095s 7s/step - accuracy: 0.6814 - loss: 0.7830 - val_accuracy: 0.6945 - val_loss: 0.7645


#### Evaluasi Skema Pelatihan 2

In [10]:
y_pred2 = model2.predict(X_test_tfidf_3d)
y_pred2_classes = np.argmax(y_pred2, axis=1)

print("Accuracy Skema 2:", accuracy_score(y_test, y_pred2_classes))
print(classification_report(y_test, y_pred2_classes, target_names=le.classes_))

625/625 ━━━━━━━━━━━━━━━━━━━━ 210s 335ms/step
Accuracy Skema 2: 0.69455
              precision    recall  f1-score   support

    negative       0.75      0.11      0.20      5200
     neutral       0.00      0.00      0.00      1300
    positive       0.69      0.99      0.81     13500

    accuracy                           0.69     20000
   macro avg       0.48      0.37      0.34     20000
weighted avg       0.66      0.69      0.60     20000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Skema Pelatihan 3: CNN + Word2Vec + Split 70/30

In [11]:
# Split ulang 70/30
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    df['clean_text'], df['label_encoded'],
    test_size=0.3, stratify=df['label_encoded'], random_state=42
)

# Tokenizing ulang
X_train2_seq = tokenizer.texts_to_sequences(X_train2)
X_test2_seq = tokenizer.texts_to_sequences(X_test2)

X_train2_pad = pad_sequences(X_train2_seq, maxlen=MAX_SEQUENCE_LENGTH)
X_test2_pad = pad_sequences(X_test2_seq, maxlen=MAX_SEQUENCE_LENGTH)

y_train2_cat = to_categorical(y_train2, num_classes=3)
y_test2_cat = to_categorical(y_test2, num_classes=3)

# CNN model
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

model3 = Sequential()
model3.add(Embedding(MAX_NUM_WORDS, 100, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model3.add(Conv1D(128, 5, activation='relu'))
model3.add(GlobalMaxPooling1D())
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.3))
model3.add(Dense(3, activation='softmax'))

model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history3 = model3.fit(X_train2_pad, y_train2_cat, epochs=5, batch_size=512, validation_data=(X_test2_pad, y_test2_cat))


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


137/137 ━━━━━━━━━━━━━━━━━━━━ 58s 414ms/step - accuracy: 0.9033 - loss: 0.3283 - val_accuracy: 0.9600 - val_loss: 0.1032
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 80s 400ms/step - accuracy: 0.9589 - loss: 0.1071 - val_accuracy: 0.9600 - val_loss: 0.1019
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 50s 367ms/step - accuracy: 0.9615 - loss: 0.1017 - val_accuracy: 0.9649 - val_loss: 0.1015
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 86s 399ms/step - accuracy: 0.9610 - loss: 0.1023 - val_accuracy: 0.9600 - val_loss: 0.1015
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 84s 412ms/step - accuracy: 0.9614 - loss: 0.1016 - val_accuracy: 0.9600 - val_loss: 0.0999


#### Evaluasi Skema Pelatihan 3

In [12]:
y_pred3 = model3.predict(X_test2_pad)
y_pred3_classes = np.argmax(y_pred3, axis=1)

print("Accuracy Skema 3:", accuracy_score(y_test2, y_pred3_classes))
print(classification_report(y_test2, y_pred3_classes, target_names=le.classes_))


938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step
Accuracy Skema 3: 0.9599666666666666
              precision    recall  f1-score   support

    negative       1.00      0.87      0.93      7800
     neutral       1.00      0.92      0.96      1950
    positive       0.94      1.00      0.97     20250

    accuracy                           0.96     30000
   macro avg       0.98      0.93      0.95     30000
weighted avg       0.96      0.96      0.96     30000



## Langkah 7 Inference Manual (Testing)


In [13]:
def predict_sentiment(text):
    clean = remove_stopwords(clean_text(text))
    seq = tokenizer.texts_to_sequences([clean])
    pad = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model1.predict(pad)
    return le.inverse_transform([np.argmax(pred)])

# Contoh
predict_sentiment("This app is amazing and fun to use!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 726ms/step


array(['positive'], dtype=object)

In [14]:
!pip install pipreqs
!pipreqs /content --force

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d5d2dd069ca69093fdcf39446d4464d13284473cf306c72b79b2174a42199570
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.12.3 which is incompatible.


INFO: Not scanning for jupyter notebooks.
INFO: Successfully saved requirements file in /content/requirements.txt
